# Path of documents

In [ ]:
# Folder containing .txt files
#Here just put the path where are yur documents!
folder_path = r'C:/Users/mateo/OneDrive/Documentos/IMT NORD EUROPE/AÑO 2/Logistics and sim/Project'


# CPLEX Solution

In [ ]:
import os
import pandas as pd
from docplex.mp.context import Context
from docplex.mp.model import Model

# Explicitly set the CPLEX path to use the full version
custom_context = Context.make_default_context()
custom_context.solver.cplex_path = r"C:\Program Files\IBM\ILOG\CPLEX_Studio2211\cplex\bin\x64_win64\cplex.exe"

# Confirm the CPLEX path
print(f"CPLEX Path: {custom_context.solver.cplex_path}")


# Function to solve the parallel machine scheduling problem
def solve_parallel_scheduling(m, n, processing_times, time_limit=150):
    # Initialize the model with the custom context
    mdl = Model(name="Parallel_Machine_Scheduling", context=custom_context)
    mdl.context.cplex_parameters.timelimit = time_limit

    # Decision variables
    x = mdl.binary_var_matrix(keys1=range(n), keys2=range(m), name="x")
    Cmax = mdl.continuous_var(name="Cmax")

    # Objective: Minimize Cmax
    mdl.minimize(Cmax)

    # Constraints: Each task assigned to exactly one machine
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[i, j] for j in range(m)) == 1, ctname=f"task_{i}_assignment")
    
    # Constraints: Machine workloads cannot exceed Cmax
    for j in range(m):
        mdl.add_constraint(
            mdl.sum(processing_times[i] * x[i, j] for i in range(n)) <= Cmax,
            ctname=f"machine_{j}_load"
        )

    # Solve the model
    solution = mdl.solve(log_output=True)

    # Collect results
    if solution:
        return {
            'Optimal Makespan': solution.objective_value,
            'Task Assignment': [[int(x[i, j].solution_value) for j in range(m)] for i in range(n)],
            'Lower Bound (LB)': mdl.solve_details.best_bound,
            'Gap': mdl.solve_details.mip_relative_gap,
            'Execution Time': mdl.solve_details.time
        }
    else:
        return {
            'Optimal Makespan': None,
            'Task Assignment': None,
            'Lower Bound (LB)': None,
            'Gap': None,
            'Execution Time': None
        }

# Process all .txt files in the folder
results = []

def process_scheduling_files(folder_path, output_csv_path):
    results = []

    # Load existing results if the CSV file exists
    if os.path.exists(output_csv_path):
        processed_df = pd.read_csv(output_csv_path)
        processed_files = set(processed_df['File'].tolist())
    else:
        processed_files = set()

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt') and file_name not in processed_files:
            try:
                file_path = os.path.join(folder_path, file_name)

                print(f"Processing file: {file_name}")

                # Read the file
                with open(file_path, 'r') as file:
                    lines = file.readlines()
                    m = int(lines[0].strip())  # Number of machines
                    n = int(lines[1].strip())  # Number of tasks
                    processing_times = list(map(int, lines[2].strip().split()))

                
                metrics = solve_parallel_scheduling(m, n, processing_times, time_limit=150)

                
                result = {
                    'File': file_name,
                    'Machines': m,
                    'Tasks': n,
                    'Processing Times': processing_times,
                    **metrics
                }

                results.append(result)

                
                result_df = pd.DataFrame([result])
                if os.path.exists(output_csv_path):
                    result_df.to_csv(output_csv_path, mode='a', index=False, header=False)
                else:
                    result_df.to_csv(output_csv_path, index=False)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    
    if results:
        final_df = pd.DataFrame(results)
        return final_df
    else:
        return pd.DataFrame()

#folder_path = "C:/Users/mateo/opl/Projet1/INSTANCES/INSTANCES"
output_csv_path = "C:/Users/mateo/OneDrive/Documentos/IMT NORD EUROPE/AÑO 2/Logistics and sim/scheduling_results_cplex.csv"
results_df = process_scheduling_files(folder_path, output_csv_path)


print(results_df)


CPLEX Path: C:\Program Files\IBM\ILOG\CPLEX_Studio2211\cplex\bin\x64_win64\cplex.exe
Processing file: U_2_0050_25_3.txt
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               150
Found incumbent of value 26639.000000 after 0.02 sec. (0.08 ticks)
Tried aggregator 1 time.
Reduced MIP has 75 rows, 1251 columns, and 2525 nonzeros.
Reduced MIP has 1250 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (1.74 ticks)
Probing time = 0.00 sec. (2.21 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 75 rows, 1251 columns, and 2525 nonzeros.
Reduced MIP has 1250 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (4.03 ticks)
Probing time = 0.00 sec. (2.20 ticks)
Clique table members: 50.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root rela

In [5]:
results_cplex_df = results_df.copy()
#results_df.to_excel('scheduling_results_cplex.xlsx')

# Heuristic models

## Funtion for the heuristic forms

In [ ]:
import os
import pandas as pd
import time

def calculate_cmax_lpt(machines, tasks, process_times):
    
    # Sort tasks in descending order of processing time
    sorted_tasks = sorted(process_times, reverse=True)
    # Initialize machine workloads
    machine_workloads = [0] * machines

    for task in sorted_tasks:
        # Assign the task to the machine with the least workload
        min_machine = machine_workloads.index(min(machine_workloads))
        machine_workloads[min_machine] += task

    # The maximum workload is the Cmax
    return max(machine_workloads)

def calculate_lower_bound(machines, process_times):

    return max(max(process_times), sum(process_times) / machines)

def process_files(folder_path):
    
    results = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'r') as file:
                lines = file.readlines()

                # Parse the input
                machines = int(lines[0].strip())
                tasks = int(lines[1].strip())
                process_times = list(map(int, lines[2].split()))

                # Validate input
                if len(process_times) != tasks:
                    raise ValueError(f"Mismatch in number of tasks for file {file_name}")

                # Start timing the calculation
                start_time = time.time()

                # Calculate metrics
                cmax = calculate_cmax_lpt(machines, tasks, process_times)
                lower_bound = calculate_lower_bound(machines, process_times)
                processing_time = time.time() - start_time

                # Append results
                results.append({
                    'Document Name': file_name,
                    'm': machines,
                    'n': tasks,
                    'pj': process_times,
                    'Objective Value (Cmax)': cmax,
                    'Lower Bound': lower_bound,
                    'Processing Time (s)': processing_time
                })

    # Convert results to DataFrame
    df = pd.DataFrame(results)
    return df

# Example usage
#folder_path = "C:/Users/mateo/opl/Projet1/INSTANCES/testpy"
result__LPT_df = process_files(folder_path)
result__LPT_df


,Document Name,m,n,pj,Objective Value (Cmax),Lower Bound,Processing Time (s)
0,U_3_0050_10_4.txt,10,50,"[5560, 6730, 7050, 6535, 3222, 5891, 6487, 963...",28535,27204.1,0.0


## Local research: permutation

We are gonna take the higher value and changed with the lower value

In [ ]:
import os
import pandas as pd
import time

def calculate_cmax_lpt_matrix(machines, process_times):

    sorted_tasks = sorted(process_times, reverse=True)
    machine_workloads = [0] * machines
    machine_allocation = [[] for _ in range(machines)]

    for task in sorted_tasks:
        # Assign task to the machine with the minimum workload
        min_machine = machine_workloads.index(min(machine_workloads))
        machine_workloads[min_machine] += task
        machine_allocation[min_machine].append(task)

    # Convert the allocation to a DataFrame (task matrix)
    max_tasks = max(len(machine) for machine in machine_allocation)
    matrix = pd.DataFrame(
        [machine + [0] * (max_tasks - len(machine)) for machine in machine_allocation]
    )
    matrix["C"] = machine_workloads  # Add workloads as the last column (C)

    return max(machine_workloads), matrix

def local_search_cmax_v3(matrix):
  
    best_matrix = matrix.copy()
    best_cmax = matrix['C'].max()
    columns = list(matrix.columns[:-1])  # Exclude the 'C' column

    col_index = 0  # Start with the first column
    changes_made = 0  # Counter for the number of changes

    while col_index < len(columns):  # Iterate through columns
        col = columns[col_index]

        # Identify rows with Cmax and Cmin
        cmax_index = matrix['C'].idxmax()
        cmin_index = matrix['C'].idxmin()
        cmax_value = matrix['C'].max()
        cmin_value = matrix['C'].min()

        # Swap values in the current column
        matrix.at[cmax_index, col], matrix.at[cmin_index, col] = (
            matrix.at[cmin_index, col],
            matrix.at[cmax_index, col],
        )

        # Recalculate C for all rows
        matrix['C'] = matrix.drop(columns='C').sum(axis=1)

        # Check new Cmax
        new_cmax = matrix['C'].max()

        if new_cmax < best_cmax:  # If improved, stay in the same column
            best_cmax = new_cmax
            best_matrix = matrix.copy()
            changes_made += 1  # Count the change
            col_index = 0  # Restart from the first column

         
        else:  # If no improvement, revert and move to the next column
            matrix = best_matrix.copy()
            col_index += 1  # Move to the next column
 


    return best_cmax, best_matrix, changes_made



def process_files_with_local_search_v3(folder_path):

    results = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'r') as file:
                lines = file.readlines()

                # Parse the input
                machines = int(lines[0].strip())
                tasks = int(lines[1].strip())
                process_times = list(map(int, lines[2].split()))

                # Validate input
                if len(process_times) != tasks:
                    raise ValueError(f"Mismatch in number of tasks for file {file_name}")

                # Start timing the calculation
                start_time = time.time()

                # Calculate initial metrics and matrix
                cmax, initial_matrix = calculate_cmax_lpt_matrix(machines, process_times)

                # Perform local search
                improved_cmax, improved_matrix,changes_done = local_search_cmax_v3(initial_matrix.copy())

                # Processing time
                processing_time = time.time() - start_time

                # Append results
                results.append({
                    'Document Name': file_name,
                    'm': machines,
                    'n': tasks,
                    'pj': process_times,
                    'Objective Value (Cmax)': cmax,
                    'Improved Cmax (Local Search)': improved_cmax,
                    'Local research movements' : changes_done,
                    'Processing Time (s)': processing_time,
                    'Initial Matrix': initial_matrix,
                    'Final Matrix': improved_matrix
                })

    # Convert results to DataFrame
    df = pd.DataFrame(results)
    return df


# Example usage
#folder_path = "C:/Users/mateo/opl/Projet1/INSTANCES/testpy"
result_simple_decend_df = process_files_with_local_search_v3(folder_path)


In [ ]:
result_simple_decend_df

,Document Name,m,n,pj,Objective Value (Cmax),Improved Cmax (Local Search),Local research movements,Processing Time (s),Initial Matrix,Final Matrix
0,U_3_0050_10_4.txt,10,50,"[5560, 6730, 7050, 6535, 3222, 5891, 6487, 963...",28535,27741,8,0.158067,0 1 2 3 4 5 C 0 ...,0 1 2 3 4 5 C 0 ...


## Deepest decend method


In [ ]:
import os
import pandas as pd
import time
from itertools import permutations

def calculate_cmax_lpt_matrix(machines, process_times):
    sorted_tasks = sorted(process_times, reverse=True)
    machine_workloads = [0] * machines
    machine_allocation = [[] for _ in range(machines)]

    for task in sorted_tasks:
        # Assign task to the machine with the minimum workload
        min_machine = machine_workloads.index(min(machine_workloads))
        machine_workloads[min_machine] += task
        machine_allocation[min_machine].append(task)

    # Convert the allocation to a DataFrame (task matrix)
    max_tasks = max(len(machine) for machine in machine_allocation)
    matrix = pd.DataFrame(
        [machine + [0] * (max_tasks - len(machine)) for machine in machine_allocation]
    )
    matrix["C"] = machine_workloads  # Add workloads as the last column (C)

    return max(machine_workloads), matrix


def local_search_cmax_v3(matrix):
    best_matrix = matrix.copy()
    best_cmax = matrix['C'].max()
    columns = list(matrix.columns[:-1])  # Exclude the 'C' column

    col_index = 0  # Start with the first column
    changes_made = 0  # Counter for the number of changes

    while col_index < len(columns):  # Iterate through columns
        col = columns[col_index]

        # Identify rows with Cmax and Cmin
        cmax_index = matrix['C'].idxmax()
        cmin_index = matrix['C'].idxmin()

        # Swap values in the current column
        matrix.at[cmax_index, col], matrix.at[cmin_index, col] = (
            matrix.at[cmin_index, col],
            matrix.at[cmax_index, col],
        )

        # Recalculate C for all rows
        matrix['C'] = matrix.drop(columns='C').sum(axis=1)

        # Check new Cmax
        new_cmax = matrix['C'].max()

        if new_cmax < best_cmax:  # If improved, stay in the same column
            best_cmax = new_cmax
            best_matrix = matrix.copy()
            changes_made += 1  # Count the change
            col_index = 0  # Restart from the first column
        else:  # If no improvement, revert and move to the next column
            matrix = best_matrix.copy()
            col_index += 1  # Move to the next column

    return best_cmax, best_matrix, changes_made


def deepest(machines, planning):
    def calculate_workloads(planning):
        """Calculate workloads (C) for each machine."""
        return [sum(machine) for machine in planning]

    def find_extremes(workloads):
        """Identify indices of the machines with Cmax and Cmin."""
        max_idx = workloads.index(max(workloads))
        min_idx = workloads.index(min(workloads))
        return max_idx, min_idx

    def test_permutations(max_machine, min_machine, Cmax):
        """Test all permutations of tasks between two machines."""
        best_permutation = None
        best_Cmax = Cmax
        for i in range(min(len(max_machine), len(min_machine))):
            # Swap tasks at position i
            new_max_machine = max_machine[:]
            new_min_machine = min_machine[:]
            new_max_machine[i], new_min_machine[i] = new_min_machine[i], new_max_machine[i]

            # Calculate new workloads
            new_Cmax_machine = sum(new_max_machine)
            new_Cmin_machine = sum(new_min_machine)

            # Check if the new workloads are valid
            if new_Cmax_machine <= Cmax and new_Cmin_machine <= Cmax:
                new_overall_Cmax = max(new_Cmax_machine, new_Cmin_machine)
                if new_overall_Cmax < best_Cmax:
                    best_Cmax = new_overall_Cmax
                    best_permutation = (i, new_max_machine, new_min_machine)

        return best_permutation

    workloads = calculate_workloads(planning)
    changes = 0
    while True:
        # Identify the machines with Cmax and Cmin
        max_idx, min_idx = find_extremes(workloads)
        Cmax = workloads[max_idx]

        # Test all permutations
        best_permutation = test_permutations(planning[max_idx], planning[min_idx], Cmax)

        if not best_permutation:
            # No valid swaps, stop the algorithm
            break

        # Apply the best permutation found
        i, new_max_machine, new_min_machine = best_permutation
        planning[max_idx] = new_max_machine
        planning[min_idx] = new_min_machine

        # Recalculate workloads
        workloads = calculate_workloads(planning)
        changes += 1

    # Return the final Cmax and planning
    return max(workloads), planning, changes


def process_files_with_local_search_v3(folder_path):
    results = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'r') as file:
                lines = file.readlines()

                # Parse the input
                machines = int(lines[0].strip())
                tasks = int(lines[1].strip())
                process_times = list(map(int, lines[2].split()))

                # Validate input
                if len(process_times) != tasks:
                    raise ValueError(f"Mismatch in number of tasks for file {file_name}")

                # Calculate initial metrics and matrix
                cmax, initial_matrix = calculate_cmax_lpt_matrix(machines, process_times)

                # Measure time for LPT-based local search
                start_time_lpt = time.time()
                improved_cmax_lpt, improved_matrix_lpt, changes_done_lpt = local_search_cmax_v3(initial_matrix.copy())
                processing_time_lpt = time.time() - start_time_lpt

                # Prepare data for the deepest algorithm
                planning = [list(row.dropna()) for _, row in initial_matrix.iloc[:, :-1].iterrows()]

                # Measure time for deepest algorithm
                start_time_deep = time.perf_counter()
                improved_cmax_deep, improved_planning, changes_done_deep = deepest(machines, planning)
                processing_time_deep = time.perf_counter() - start_time_deep
                
                # Append results
                results.append({
                    'Document Name': file_name,
                    'm': machines,
                    'n': tasks,
                    'pj': process_times,
                    'Initial Cmax by LPT': cmax,
                    'Simple decend Improved Cmax': improved_cmax_lpt,
                    'Deepest Improved Cmax': improved_cmax_deep,
                    'Simple decend Changes': changes_done_lpt,
                    'Deepest Changes': changes_done_deep,
                    'Simple decend Processing Time (s)': processing_time_lpt,
                    'Deepest Processing Time (s)': processing_time_deep,
                    #'Initial Matrix': initial_matrix,
                    #'LPT Final Matrix': improved_matrix_lpt,
                    #'Deepest Final Planning': improved_planning
                })

    # Convert results to DataFrame
    df = pd.DataFrame(results)
    return df


#folder_path = "C:/Users/mateo/opl/Projet1/INSTANCES/INSTANCES"
result_deepest_df = process_files_with_local_search_v3(folder_path)

result_deepest_df

,Document Name,m,n,pj,Initial Cmax by LPT,Simple decend Improved Cmax,Deepest Improved Cmax,Simple decend Changes,Deepest Changes,Simple decend Processing Time (s),Deepest Processing Time (s)
0,NU_1_0010_05_0.txt,5,10,"[99, 90, 96, 98, 96, 95, 98, 97, 95, 1]",193,193,193,0,0,0.055547,0.000022
1,NU_1_0010_05_1.txt,5,10,"[97, 90, 99, 90, 98, 91, 100, 94, 93, 15]",189,189,189,0,0,0.016482,0.000020
2,NU_1_0010_05_2.txt,5,10,"[92, 91, 94, 95, 92, 94, 100, 93, 90, 3]",186,186,186,0,0,0.011248,0.000039
3,NU_1_0010_05_3.txt,5,10,"[97, 97, 91, 93, 92, 91, 97, 90, 91, 7]",188,188,188,0,0,0.010342,0.000027
4,NU_1_0010_05_4.txt,5,10,"[91, 97, 98, 93, 94, 97, 95, 95, 100, 10]",191,191,191,0,0,0.009769,0.000034
...,...,...,...,...,...,...,...,...,...,...,...
775,U_3_1000_25_5.txt,25,1000,"[2825, 1424, 2153, 1045, 1248, 6998, 6315, 714...",203833,203830,203792,1,16,0.064060,0.001071
776,U_3_1000_25_6.txt,25,1000,"[6070, 1068, 3664, 4120, 4815, 6534, 2784, 354...",195535,195531,195490,2,12,0.053054,0.000783
777,U_3_1000_25_7.txt,25,1000,"[954, 8334, 5250, 7696, 2167, 2978, 2378, 5871...",201153,201148,201148,4,4,0.057003,0.000315
778,U_3_1000_25_8.txt,25,1000,"[8774, 771, 9923, 1683, 2788, 1148, 3175, 533,...",202513,202513,202469,0,17,0.064011,0.001069


In [37]:
result_deepest_df.to_excel('results_time_deepest.xlsx')

In [36]:
row_count = result_deepest_df[result_deepest_df['Deepest Processing Time (s)'] == 0].shape[0]
row_count


0

# Metha-heuristics

## Multi - Start

In [ ]:
import random
import os
import pandas as pd
import time
from itertools import permutations

def calculate_cmax_lpt_matrix(machines, process_times):
    sorted_tasks = sorted(process_times, reverse=True)
    machine_workloads = [0] * machines
    machine_allocation = [[] for _ in range(machines)]

    for task in sorted_tasks:
        # Assign task to the machine with the minimum workload
        min_machine = machine_workloads.index(min(machine_workloads))
        machine_workloads[min_machine] += task
        machine_allocation[min_machine].append(task)

    # Convert the allocation to a DataFrame (task matrix)
    max_tasks = max(len(machine) for machine in machine_allocation)
    matrix = pd.DataFrame(
        [machine + [0] * (max_tasks - len(machine)) for machine in machine_allocation]
    )
    matrix["C"] = machine_workloads  # Add workloads as the last column (C)

    return max(machine_workloads), matrix


def local_search_cmax_v3(matrix):
    best_matrix = matrix.copy()
    best_cmax = matrix['C'].max()
    columns = list(matrix.columns[:-1])  # Exclude the 'C' column

    col_index = 0  # Start with the first column
    changes_made = 0  # Counter for the number of changes

    while col_index < len(columns):  # Iterate through columns
        col = columns[col_index]

        # Identify rows with Cmax and Cmin
        cmax_index = matrix['C'].idxmax()
        cmin_index = matrix['C'].idxmin()

        # Swap values in the current column
        matrix.at[cmax_index, col], matrix.at[cmin_index, col] = (
            matrix.at[cmin_index, col],
            matrix.at[cmax_index, col],
        )

        # Recalculate C for all rows
        matrix['C'] = matrix.drop(columns='C').sum(axis=1)

        # Check new Cmax
        new_cmax = matrix['C'].max()

        if new_cmax < best_cmax:  # If improved, stay in the same column
            best_cmax = new_cmax
            best_matrix = matrix.copy()
            changes_made += 1  # Count the change
            col_index = 0  # Restart from the first column
        else:  # If no improvement, revert and move to the next column
            matrix = best_matrix.copy()
            col_index += 1  # Move to the next column

    return best_cmax, best_matrix, changes_made


def deepest(machines, planning):
    def calculate_workloads(planning):

        return [sum(machine) for machine in planning]

    def find_extremes(workloads):

        max_idx = workloads.index(max(workloads))
        min_idx = workloads.index(min(workloads))
        return max_idx, min_idx

    def test_permutations(max_machine, min_machine, Cmax):

        best_permutation = None
        best_Cmax = Cmax
        for i in range(min(len(max_machine), len(min_machine))):
            # Swap tasks at position i
            new_max_machine = max_machine[:]
            new_min_machine = min_machine[:]
            new_max_machine[i], new_min_machine[i] = new_min_machine[i], new_max_machine[i]

            # Calculate new workloads
            new_Cmax_machine = sum(new_max_machine)
            new_Cmin_machine = sum(new_min_machine)

            # Check if the new workloads are valid
            if new_Cmax_machine <= Cmax and new_Cmin_machine <= Cmax:
                new_overall_Cmax = max(new_Cmax_machine, new_Cmin_machine)
                if new_overall_Cmax < best_Cmax:
                    best_Cmax = new_overall_Cmax
                    best_permutation = (i, new_max_machine, new_min_machine)

        return best_permutation

    workloads = calculate_workloads(planning)
    changes = 0
    while True:
        # Identify the machines with Cmax and Cmin
        max_idx, min_idx = find_extremes(workloads)
        Cmax = workloads[max_idx]

        # Test all permutations
        best_permutation = test_permutations(planning[max_idx], planning[min_idx], Cmax)

        if not best_permutation:
            # No valid swaps, stop the algorithm
            break

        # Apply the best permutation found
        i, new_max_machine, new_min_machine = best_permutation
        planning[max_idx] = new_max_machine
        planning[min_idx] = new_min_machine

        # Recalculate workloads
        workloads = calculate_workloads(planning)
        changes += 1

    # Return the final Cmax and planning as DataFrame
    return max(workloads), pd.DataFrame(planning), changes


def random_initial_solution(machines, process_times):
    random.shuffle(process_times)
    _, matrix = calculate_cmax_lpt_matrix(machines, process_times)
    return matrix


def multi_start_search_with_variants(machines, process_times, num_starts=10):
    best_cmax = float('inf')
    best_matrix = None
    best_method = None
    best_strategy = None
    best_changes = 0
    total_processing_time = 0

    results = []

    # Add the initial matrices based on LPT and SPT
    start_matrices = []
    cmax_lpt, lpt_matrix = calculate_cmax_lpt_matrix(machines, process_times)
    start_matrices.append(('LPT', lpt_matrix))

    sorted_process_times = sorted(process_times)
    cmax_spt, spt_matrix = calculate_cmax_lpt_matrix(machines, sorted_process_times)
    start_matrices.append(('SPT', spt_matrix))

    # Random starts
    for _ in range(num_starts - 2):
        random_matrix = random_initial_solution(machines, process_times)
        start_matrices.append(('Random', random_matrix))

    for method, initial_matrix in start_matrices:
        # Track the local search
        start_time = time.time()
        cmax_local, matrix_local, changes_local = local_search_cmax_v3(initial_matrix.copy())
        processing_time_local = time.time() - start_time
        results.append({
            'method': method,
            'strategy': 'Local Search',
            'cmax': cmax_local,
            'matrix': matrix_local,
            'changes': changes_local,
            'processing_time': processing_time_local,
        })


        planning_matrix = [list(row.dropna()) for _, row in initial_matrix.iloc[:, :-1].iterrows()]
        # Track the deepest search
        start_time = time.time()
        cmax_deepest, matrix_deepest, changes_deepest = deepest(machines, planning_matrix)
        processing_time_deepest = time.time() - start_time
        results.append({
            'method': method,
            'strategy': 'Deepest',
            'cmax': cmax_deepest,
            'matrix': matrix_deepest,
            'changes': changes_deepest,
            'processing_time': processing_time_deepest,
        })

        # Compare Local Search results
        if cmax_local < best_cmax:
            best_cmax = cmax_local
            best_matrix = matrix_local
            best_method = method
            best_strategy = 'Local Search'
            best_changes = changes_local

        # Compare Deepest results
        if cmax_deepest < best_cmax:
            best_cmax = cmax_deepest
            best_matrix = pd.DataFrame(matrix_deepest)  # Convert back to DataFrame
            best_method = method
            best_strategy = 'Deepest'
            best_changes = changes_deepest

        total_processing_time += processing_time_local + processing_time_deepest

    return {
        'best_cmax': best_cmax,
        'best_matrix': best_matrix,
        'best_method': best_method,
        'best_strategy': best_strategy,
        'best_changes': best_changes,
        'total_processing_time': total_processing_time,
        'all_results': results,
    }


def process_files_with_local_search_v3(folder_path):
    results = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'r') as file:
                lines = file.readlines()

                # Parse the input
                machines = int(lines[0].strip())
                tasks = int(lines[1].strip())
                process_times = list(map(int, lines[2].split()))

                # Validate input
                if len(process_times) != tasks:
                    raise ValueError(f"Mismatch in number of tasks for file {file_name}")

                # Calculate initial metrics and matrix
                cmax, initial_matrix = calculate_cmax_lpt_matrix(machines, process_times)

                # Measure time for LPT-based local search
                start_time_lpt = time.time()
                improved_cmax_lpt, improved_matrix_lpt, changes_done_lpt = local_search_cmax_v3(initial_matrix.copy())
                processing_time_lpt = time.time() - start_time_lpt

                # Prepare data for the deepest algorithm
                planning = [list(row.dropna()) for _, row in initial_matrix.iloc[:, :-1].iterrows()]

                # Measure time for deepest algorithm
                start_time_deep = time.time()
                improved_cmax_deep, improved_planning, changes_done_deep = deepest(machines, planning)
                processing_time_deep = time.time() - start_time_deep

                # Run the multi-start search
                start_time = time.time()
                multi_start_results = multi_start_search_with_variants(machines, process_times)
                processing_time = time.time() - start_time
                
                results.append({                
                    'Document Name': file_name,
                    'm': machines,
                    'n': tasks,
                    'pj': process_times,
                    'Initial Cmax by LPT': cmax,
                    'Simple decend Improved Cmax': improved_cmax_lpt,
                    'Deepest Improved Cmax': improved_cmax_deep,
                    'Simple decend Changes': changes_done_lpt,
                    'Deepest Changes': changes_done_deep,
                    'Simple decend Processing Time (s)': processing_time_lpt,
                    'Deepest Processing Time (s)': processing_time_deep,
                    'Best Cmax': multi_start_results['best_cmax'],
                    'Best Method': multi_start_results['best_method'],
                    'Best Strategy': multi_start_results['best_strategy'],
                    'Best Changes': multi_start_results['best_changes'],
                    'Total Processing Time in multi-start': multi_start_results['total_processing_time'],
                })

    # Convert results to DataFrame
    df = pd.DataFrame(results)
    return df


# Example usage
#folder_path = "C:/Users/mateo/opl/Projet1/INSTANCES/testpy"
result_multi_start_df = process_files_with_local_search_v3(folder_path)

# Display results
result_multi_start_df


,Document Name,m,n,pj,Initial Cmax by LPT,Simple decend Improved Cmax,Deepest Improved Cmax,Simple decend Changes,Deepest Changes,Simple decend Processing Time (s),Deepest Processing Time (s),Best Cmax,Best Method,Best Strategy,Best Changes,Total Processing Time in multi-start
0,U_3_0050_10_4.txt,10,50,"[9415, 2153, 5560, 9899, 6535, 9445, 6857, 980...",28535,27741,27480,8,7,0.086409,0.00209,27480,LPT,Deepest,7,0.47996


## Tabu Method

In [ ]:
def tabu_search(machines, process_times, tabu_tenure=5, max_iterations=100):
    
    # Initialize variables
    best_cmax, best_matrix = calculate_cmax_lpt_matrix(machines, process_times)
    current_cmax, current_matrix = best_cmax, best_matrix.copy()
    tabu_list = []
    iteration = 0
    no_improvement_count = 0

    def matrix_to_tuple(matrix):

        return tuple(tuple(row) for row in matrix.iloc[:, :-1].values)

    def generate_neighbors(matrix):

        neighbors = []
        rows, cols = matrix.shape[0], matrix.shape[1] - 1  # Exclude the last column (Cmax)
        for i in range(rows):
            for j in range(i + 1, rows):
                for col in range(cols):
                    neighbor = matrix.copy()
                    # Swap tasks between two rows in the same column
                    neighbor.iloc[i, col], neighbor.iloc[j, col] = neighbor.iloc[j, col], neighbor.iloc[i, col]
                    neighbors.append(neighbor)
        return neighbors

    while iteration < max_iterations and no_improvement_count < 10:
        neighbors = generate_neighbors(current_matrix)
        best_neighbor = None
        best_neighbor_cmax = float('inf')

        for neighbor in neighbors:
            # Convert the neighbor to a hashable type
            neighbor_tuple = matrix_to_tuple(neighbor)

            # Check if neighbor is not in the Tabu list
            if neighbor_tuple not in tabu_list:
                neighbor_cmax = calculate_cmax(neighbor)
                if neighbor_cmax < best_neighbor_cmax:
                    best_neighbor = neighbor
                    best_neighbor_cmax = neighbor_cmax

        if best_neighbor is None:
            break  # No valid moves, terminate the search

        # Update current solution
        current_cmax, current_matrix = best_neighbor_cmax, best_neighbor.copy()

        # Update the global best if improved
        if best_neighbor_cmax < best_cmax:
            best_cmax, best_matrix = best_neighbor_cmax, best_neighbor.copy()
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        # Update Tabu list
        tabu_list.append(matrix_to_tuple(current_matrix))
        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

        iteration += 1

    return best_cmax, best_matrix


def process_files_with_tabu_search(folder_path, output_csv):
    results = []

    # Load existing results if the CSV file exists
    if os.path.exists(output_csv):
        processed_df = pd.read_csv(output_csv)
        processed_files = set(processed_df['Document Name'].tolist())
    else:
        processed_files = set()

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt') and file_name not in processed_files:
            try:
                file_path = os.path.join(folder_path, file_name)

                with open(file_path, 'r') as file:
                    lines = file.readlines()

                    # Parse the input
                    machines = int(lines[0].strip())
                    tasks = int(lines[1].strip())
                    process_times = list(map(int, lines[2].split()))

                    print(file_name, machines, tasks, process_times)

                    # Validate input
                    if len(process_times) != tasks:
                        raise ValueError(f"Mismatch in number of tasks for file {file_name}")

                    # Calculate initial metrics and matrix
                    cmax, initial_matrix = calculate_cmax_lpt_matrix(machines, process_times)
                    print('Initial Cmax by LPT: ', cmax)

                    # Measure time for LPT-based local search
                    start_time_lpt = time.time()
                    improved_cmax_lpt, improved_matrix_lpt, changes_done_lpt = local_search_cmax_v3(initial_matrix.copy())
                    processing_time_lpt = time.time() - start_time_lpt

                    print('Simple descend Improved Cmax: ', improved_cmax_lpt, 'Time simple descend: ', processing_time_lpt)

                    # Prepare data for the deepest algorithm
                    planning = [list(row.dropna()) for _, row in initial_matrix.iloc[:, :-1].iterrows()]

                    # Measure time for deepest algorithm
                    start_time_deep = time.time()
                    improved_cmax_deep, improved_planning, changes_done_deep = deepest(machines, planning)
                    processing_time_deep = time.time() - start_time_deep

                    print('Deepest descend Improved Cmax: ', improved_cmax_deep, 'Time deepest descend: ', processing_time_deep)

                    # Run the multi-start search
                    start_time_multi = time.time()
                    multi_start_results = multi_start_search_with_variants(machines, process_times)
                    processing_time_multi = time.time() - start_time_multi

                    print('Best Cmax Multi-start: ', multi_start_results['best_cmax'], 'Best Method Multi-start: ',
                          multi_start_results['best_method'], 'Best Strategy Multi-start: ',
                          multi_start_results['best_strategy'], '# Changes Multi-start: ',
                          multi_start_results['best_changes'], 'Total Processing Time in multi-start: ',
                          multi_start_results['total_processing_time'])

                    # Run the Tabu Search
                    start_time_tabu = time.time()
                    tabu_cmax, tabu_matrix = tabu_search(machines, process_times)
                    processing_time_tabu = time.time() - start_time_tabu

                    print('Tabu Search Improved Cmax: ', tabu_cmax, 'Time Tabu search: ', processing_time_tabu)

                    # Append results to the final output
                    result = {
                        'Document Name': file_name,
                        'm': machines,
                        'n': tasks,
                        # 'pj': process_times,
                        'Initial Cmax by LPT': cmax,
                        'Simple descend Improved Cmax': improved_cmax_lpt,
                        'Deepest Improved Cmax': improved_cmax_deep,
                        'Simple descend # Changes': changes_done_lpt,
                        'Deepest # Changes': changes_done_deep,
                        'Simple descend Processing Time (s)': processing_time_lpt,
                        'Deepest Processing Time (s)': processing_time_deep,
                        'Tabu Search Improved Cmax': tabu_cmax,
                        'Tabu Search Processing Time (s)': processing_time_tabu,
                        'Best Cmax Multi-start': multi_start_results['best_cmax'],
                        'Best Method Multi-start': multi_start_results['best_method'],
                        'Best Strategy Multi-start': multi_start_results['best_strategy'],
                        '# Changes Multi-start': multi_start_results['best_changes'],
                        'Total Processing Time in multi-start': multi_start_results['total_processing_time'],
                    }

                    results.append(result)

                    # Save this file's result immediately to the CSV
                    result_df = pd.DataFrame([result])
                    if os.path.exists(output_csv):
                        result_df.to_csv(output_csv, mode='a', index=False, header=False)
                    else:
                        result_df.to_csv(output_csv, index=False)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

    # Final DataFrame (if needed)
    if results:
        final_df = pd.DataFrame(results)
        return final_df
    else:
        return pd.DataFrame()

# Example usage
#folder_path = "C:/Users/mateo/opl/Projet1/INSTANCES/INSTANCES"
output_csv = "processed_results_tabu.csv"
result_tabu_df = process_files_with_tabu_search(folder_path, output_csv)

# Display final results (if needed)
print(result_tabu_df)

In [7]:
results_import_df =pd.read_csv("C:/Users/mateo/OneDrive/Documentos/IMT NORD EUROPE/AÑO 2/Logistics and sim/processed_results_tabu.csv")
results_import_df

,Document Name,m,n,Initial Cmax by LPT,Simple descend Improved Cmax,Deepest Improved Cmax,Simple descend # Changes,Deepest # Changes,Simple descend Processing Time (s),Deepest Processing Time (s),Tabu Search Improved Cmax,Tabu Search Processing Time (s),Best Cmax Multi-start,Best Method Multi-start,Best Strategy Multi-start,# Changes Multi-start,Total Processing Time in multi-start
0,NU_1_0010_05_0.txt,5,10,193,193,193,0,0,0.008975,0.000997,193,0.310682,193,LPT,Local Search,0,0.065567
1,NU_1_0010_05_1.txt,5,10,189,189,189,0,0,0.003990,0.001996,189,0.773249,189,LPT,Local Search,0,0.071324
2,NU_1_0010_05_2.txt,5,10,186,186,186,0,0,0.009498,0.000997,186,0.373907,186,LPT,Local Search,0,0.085508
3,NU_1_0010_05_3.txt,5,10,188,188,188,0,0,0.002992,0.000995,188,0.246171,188,LPT,Local Search,0,0.035541
4,NU_1_0010_05_4.txt,5,10,191,191,191,0,0,0.003958,0.001038,191,0.263711,191,LPT,Local Search,0,0.034993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,U_3_1000_25_5.txt,25,1000,203833,203830,203792,1,16,0.077724,0.005853,203830,170.742764,203792,LPT,Deepest,16,0.692143
776,U_3_1000_25_6.txt,25,1000,195535,195531,195490,2,12,0.053511,0.005653,195527,218.726646,195490,LPT,Deepest,12,0.585267
777,U_3_1000_25_7.txt,25,1000,201153,201148,201148,4,4,0.063861,0.002992,201148,193.527616,201148,LPT,Local Search,4,0.634880
778,U_3_1000_25_8.txt,25,1000,202513,202513,202469,0,17,0.046459,0.003334,202513,121.643728,202469,LPT,Deepest,17,0.532952


In [9]:
results_import_df.columns

Index(['Document Name', 'm', 'n', 'Initial Cmax by LPT',
       'Simple descend Improved Cmax', 'Deepest Improved Cmax',
       'Simple descend # Changes', 'Deepest # Changes',
       'Simple descend Processing Time (s)', 'Deepest Processing Time (s)',
       'Tabu Search Improved Cmax', 'Tabu Search Processing Time (s)',
       'Best Cmax Multi-start', 'Best Method Multi-start',
       'Best Strategy Multi-start', '# Changes Multi-start',
       'Total Processing Time in multi-start'],
      dtype='object')

In [ ]:
def calculate_lower_bound(machines, process_times):
  
    return max(max(process_times), sum(process_times) / machines)

def process_files(folder_path):
 
    results = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)

            with open(file_path, 'r') as file:
                lines = file.readlines()

                # Parse the input
                machines = int(lines[0].strip())
                tasks = int(lines[1].strip())
                process_times = list(map(int, lines[2].split()))

                # Validate input
                if len(process_times) != tasks:
                    raise ValueError(f"Mismatch in number of tasks for file {file_name}")

                # Start timing the calculation
                start_time = time.time()

                # Calculate metrics
                #cmax = calculate_cmax_lpt(machines, tasks, process_times)
                lower_bound = calculate_lower_bound(machines, process_times)
                processing_time = time.time() - start_time

                # Append results
                results.append({
                    'Document Name': file_name,
                    'm': machines,
                    'n': tasks,
                    #'pj': process_times,
                    #'Objective Value (Cmax)': cmax,
                    'Lower Bound': lower_bound,
                    'Processing Time (s) LPT': processing_time
                })

    # Convert results to DataFrame
    df = pd.DataFrame(results)
    return df

# Example usage
#folder_path = "C:/Users/mateo/opl/Projet1/INSTANCES/INSTANCES"
result_df_lb = process_files(folder_path)
result_df_lb

In [ ]:
result_df_lb.to_excel('lower_bpound_cplex.xlsx')

In [10]:
lower_bound_df= pd.read_excel('lower_bpound_cplex.xlsx')
lower_bound_df

,Unnamed: 0,Document Name,m,n,Lower Bound,Processing Time (s) LPT
0,0,NU_1_0010_05_0.txt,5,10,173.00,0
1,1,NU_1_0010_05_1.txt,5,10,173.40,0
2,2,NU_1_0010_05_2.txt,5,10,168.80,0
3,3,NU_1_0010_05_3.txt,5,10,169.20,0
4,4,NU_1_0010_05_4.txt,5,10,174.00,0
...,...,...,...,...,...,...
775,775,U_3_1000_25_5.txt,25,1000,203788.56,0
776,776,U_3_1000_25_6.txt,25,1000,195468.12,0
777,777,U_3_1000_25_7.txt,25,1000,201132.28,0
778,778,U_3_1000_25_8.txt,25,1000,202464.92,0


In [11]:
results_import_df = pd.merge(results_import_df, lower_bound_df[['Document Name', 'Lower Bound']], on='Document Name', how='inner')
results_import_df

,Document Name,m,n,Initial Cmax by LPT,Simple descend Improved Cmax,Deepest Improved Cmax,Simple descend # Changes,Deepest # Changes,Simple descend Processing Time (s),Deepest Processing Time (s),Tabu Search Improved Cmax,Tabu Search Processing Time (s),Best Cmax Multi-start,Best Method Multi-start,Best Strategy Multi-start,# Changes Multi-start,Total Processing Time in multi-start,Lower Bound
0,NU_1_0010_05_0.txt,5,10,193,193,193,0,0,0.008975,0.000997,193,0.310682,193,LPT,Local Search,0,0.065567,173.00
1,NU_1_0010_05_1.txt,5,10,189,189,189,0,0,0.003990,0.001996,189,0.773249,189,LPT,Local Search,0,0.071324,173.40
2,NU_1_0010_05_2.txt,5,10,186,186,186,0,0,0.009498,0.000997,186,0.373907,186,LPT,Local Search,0,0.085508,168.80
3,NU_1_0010_05_3.txt,5,10,188,188,188,0,0,0.002992,0.000995,188,0.246171,188,LPT,Local Search,0,0.035541,169.20
4,NU_1_0010_05_4.txt,5,10,191,191,191,0,0,0.003958,0.001038,191,0.263711,191,LPT,Local Search,0,0.034993,174.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,U_3_1000_25_5.txt,25,1000,203833,203830,203792,1,16,0.077724,0.005853,203830,170.742764,203792,LPT,Deepest,16,0.692143,203788.56
776,U_3_1000_25_6.txt,25,1000,195535,195531,195490,2,12,0.053511,0.005653,195527,218.726646,195490,LPT,Deepest,12,0.585267,195468.12
777,U_3_1000_25_7.txt,25,1000,201153,201148,201148,4,4,0.063861,0.002992,201148,193.527616,201148,LPT,Local Search,4,0.634880,201132.28
778,U_3_1000_25_8.txt,25,1000,202513,202513,202469,0,17,0.046459,0.003334,202513,121.643728,202469,LPT,Deepest,17,0.532952,202464.92


In [ ]:
results_import_df['Gap LPT'] = (results_import_df['Initial Cmax by LPT'] - results_import_df['Lower Bound']) /results_import_df['Initial Cmax by LPT']
results_import_df['Gap Local Simple decend'] = (results_import_df['Simple descend Improved Cmax'] - results_import_df['Lower Bound']) /results_import_df['Simple descend Improved Cmax']
results_import_df['Gap Local Deepest decend'] = (results_import_df['Deepest Improved Cmax'] - results_import_df['Lower Bound']) /results_import_df['Deepest Improved Cmax']
results_import_df['Gap General Multi-start'] =  (results_import_df['Best Cmax Multi-start'] - results_import_df['Lower Bound']) /results_import_df['Best Cmax Multi-start']
results_import_df['Gap General Tabu Search'] = (results_import_df['Tabu Search Improved Cmax'] - results_import_df['Lower Bound']) /results_import_df['Tabu Search Improved Cmax']


In [14]:
results_import_df

,Document Name,m,n,Initial Cmax by LPT,Simple descend Improved Cmax,Deepest Improved Cmax,Simple descend # Changes,Deepest # Changes,Simple descend Processing Time (s),Deepest Processing Time (s),...,Best Method Multi-start,Best Strategy Multi-start,# Changes Multi-start,Total Processing Time in multi-start,Lower Bound,Gap LPT,Gap Local Simple decend,Gap Local Deepest decend,Gap General Multi-start,Gap General Tabu Search
0,NU_1_0010_05_0.txt,5,10,193,193,193,0,0,0.008975,0.000997,...,LPT,Local Search,0,0.065567,173.00,0.103627,0.103627,0.103627,0.103627,0.103627
1,NU_1_0010_05_1.txt,5,10,189,189,189,0,0,0.003990,0.001996,...,LPT,Local Search,0,0.071324,173.40,0.082540,0.082540,0.082540,0.082540,0.082540
2,NU_1_0010_05_2.txt,5,10,186,186,186,0,0,0.009498,0.000997,...,LPT,Local Search,0,0.085508,168.80,0.092473,0.092473,0.092473,0.092473,0.092473
3,NU_1_0010_05_3.txt,5,10,188,188,188,0,0,0.002992,0.000995,...,LPT,Local Search,0,0.035541,169.20,0.100000,0.100000,0.100000,0.100000,0.100000
4,NU_1_0010_05_4.txt,5,10,191,191,191,0,0,0.003958,0.001038,...,LPT,Local Search,0,0.034993,174.00,0.089005,0.089005,0.089005,0.089005,0.089005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,U_3_1000_25_5.txt,25,1000,203833,203830,203792,1,16,0.077724,0.005853,...,LPT,Deepest,16,0.692143,203788.56,0.000218,0.000203,0.000017,0.000017,0.000203
776,U_3_1000_25_6.txt,25,1000,195535,195531,195490,2,12,0.053511,0.005653,...,LPT,Deepest,12,0.585267,195468.12,0.000342,0.000322,0.000112,0.000112,0.000301
777,U_3_1000_25_7.txt,25,1000,201153,201148,201148,4,4,0.063861,0.002992,...,LPT,Local Search,4,0.634880,201132.28,0.000103,0.000078,0.000078,0.000078,0.000078
778,U_3_1000_25_8.txt,25,1000,202513,202513,202469,0,17,0.046459,0.003334,...,LPT,Deepest,17,0.532952,202464.92,0.000237,0.000237,0.000020,0.000020,0.000237


# Final treatment of data

In [18]:
results_cplex_df = pd.read_excel('Results_complete_cplex_heuristics.xlsx')
results_cplex_df

,File Name,Objective Value (150 sec),Obj Value (15 sec),DIFFERENCE 15-150,Percent evolution,Lower Bound,Gap(150 sec),GAP (15 sec),Processing Time(150 sec),PT 15 sec
0,NU_1_0010_05_0.txt,193,193.0,0.0,0.000000,173.00,0.000000,0.000000,0.340,0.203
1,NU_1_0010_05_1.txt,189,189.0,0.0,0.000000,173.40,0.000000,0.000000,0.090,0.094
2,NU_1_0010_05_2.txt,186,186.0,0.0,0.000000,168.80,0.000000,0.000000,0.060,0.094
3,NU_1_0010_05_3.txt,188,188.0,0.0,0.000000,169.20,0.000000,0.000000,0.080,1.484
4,NU_1_0010_05_4.txt,191,191.0,0.0,0.000000,174.00,0.000000,0.000000,0.080,0.032
...,...,...,...,...,...,...,...,...,...,...
775,U_3_1000_25_5.txt,204086,203861.0,-225.0,-0.001102,203788.56,0.001457,0.000355,150.328,15.203
776,U_3_1000_25_6.txt,195509,195535.0,26.0,0.000133,195468.12,0.000209,0.000342,150.375,15.172
777,U_3_1000_25_7.txt,201160,201246.0,86.0,0.000428,201132.28,0.000138,0.000565,150.312,15.141
778,U_3_1000_25_8.txt,202526,202506.0,-20.0,-0.000099,202464.92,0.000302,0.000203,150.422,15.094


In [19]:
total_merged_df = pd.merge(results_cplex_df, results_import_df, left_on='File Name', right_on='Document Name', how='inner')
total_merged_df

,File Name,Objective Value (150 sec),Obj Value (15 sec),DIFFERENCE 15-150,Percent evolution,Lower Bound_x,Gap(150 sec),GAP (15 sec),Processing Time(150 sec),PT 15 sec,...,Best Method Multi-start,Best Strategy Multi-start,# Changes Multi-start,Total Processing Time in multi-start,Lower Bound_y,Gap LPT,Gap Local Simple decend,Gap Local Deepest decend,Gap General Multi-start,Gap General Tabu Search
0,NU_1_0010_05_0.txt,193,193.0,0.0,0.000000,173.00,0.000000,0.000000,0.340,0.203,...,LPT,Local Search,0,0.065567,173.00,0.103627,0.103627,0.103627,0.103627,0.103627
1,NU_1_0010_05_1.txt,189,189.0,0.0,0.000000,173.40,0.000000,0.000000,0.090,0.094,...,LPT,Local Search,0,0.071324,173.40,0.082540,0.082540,0.082540,0.082540,0.082540
2,NU_1_0010_05_2.txt,186,186.0,0.0,0.000000,168.80,0.000000,0.000000,0.060,0.094,...,LPT,Local Search,0,0.085508,168.80,0.092473,0.092473,0.092473,0.092473,0.092473
3,NU_1_0010_05_3.txt,188,188.0,0.0,0.000000,169.20,0.000000,0.000000,0.080,1.484,...,LPT,Local Search,0,0.035541,169.20,0.100000,0.100000,0.100000,0.100000,0.100000
4,NU_1_0010_05_4.txt,191,191.0,0.0,0.000000,174.00,0.000000,0.000000,0.080,0.032,...,LPT,Local Search,0,0.034993,174.00,0.089005,0.089005,0.089005,0.089005,0.089005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,U_3_1000_25_5.txt,204086,203861.0,-225.0,-0.001102,203788.56,0.001457,0.000355,150.328,15.203,...,LPT,Deepest,16,0.692143,203788.56,0.000218,0.000203,0.000017,0.000017,0.000203
776,U_3_1000_25_6.txt,195509,195535.0,26.0,0.000133,195468.12,0.000209,0.000342,150.375,15.172,...,LPT,Deepest,12,0.585267,195468.12,0.000342,0.000322,0.000112,0.000112,0.000301
777,U_3_1000_25_7.txt,201160,201246.0,86.0,0.000428,201132.28,0.000138,0.000565,150.312,15.141,...,LPT,Local Search,4,0.634880,201132.28,0.000103,0.000078,0.000078,0.000078,0.000078
778,U_3_1000_25_8.txt,202526,202506.0,-20.0,-0.000099,202464.92,0.000302,0.000203,150.422,15.094,...,LPT,Deepest,17,0.532952,202464.92,0.000237,0.000237,0.000020,0.000020,0.000237


In [20]:
total_merged_df.to_excel('Final_results_project_cplex.xlsx')